# Eigenface Gesichtserkennung
* Autor: Prof. Dr. Johannes Maucher
* Datum: 27.11.2015

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.ipynb)

# Einführung
## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __Gesichtserkennung:__ mit der Eigenface Methode. 
* __Principal Component Analysis__
* __Bildverarbeitung__ mit Python.

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

Die Gesichtserkennung kann mit unterschiedlichen Ansätzen realisiert werden. In diesem Versuch wird ausschließlich der _Eigenface_-Ansatz vorgestellt. Dieser Ansatz basiert auf der _Principal Component Analysis (PCA)_ und wurde erstmals in [M. Turk, A. Pentland; Eigenfaces for Recognition](http://www.cs.ucsb.edu/%7Emturk/Papers/jcn.pdf) vorgestellt. Die Eigenface-Methode weist eine gute Performance im Fall biometrisch aufgenommener Gesichtsbilder auf.

### Das Prinzip der Eigenface Gesichtserkennung

Bilder mit $C$ Pixeln in der Breite und $R$ Pixeln in der Höhe können als $R \times C$ Matrizen abgespeichert werden. Handelt es sich um ein Schwarz-Weiß- oder Graustufen-Bild, dann wird pro Bild nur eine derartige Matrix benötigt. Der Eintrag in der i.ten Zeile und j.ten Spalte dieser Matrix definiert den Grauwert des entsprechenden Pixels. In Farbbildern werden je nach benutztem Farbraum mehrere  Matrizen pro Bild benötigt, wobei jede Matrix einen Farbkanal des Bildes repräsentiert. Für ein RGB-Bild werden z.B. 3 Matrizen für die Farbkanäle Rot, Grün und Blau benötigt. \\

Im Folgenden wird von quadratischen Graubildern mit $N \times N$ Pixeln ausgegangen. Wird jedes Pixel als ein Merkmal betrachtet, dann existieren insgesamt $N^2$ Merkmale, das Bild kann auch als ein Punkt im $N^2$-dimensionalen Raum betrachtet werden. Bilder der Auflösung $256 \times 256$ müßten also im $65536$-dimensionalen Raum beschrieben werden. Entsprechend komplex wäre die notwendige Verarbeitung. Ist jedoch bekannt, dass in einer Menge von Bildern jeweils ein gleichartiges Objekt abgebildet ist, z.B. wenn alle Bilder ausschließlich je ein Gesicht enthalten, dann existieren große Abhängigkeiten zwischen diesen Bildern. Geometrisch ausgedrückt bedeutet dies, dass die Punkte, welche die Menge der gleichartigen Bilder beschreiben, nicht gleichmäßig über den $N^2$-dimensionalen Raum verteilt sind, sondern in einen relativ kleinen Unterraum mit $K<<N^2$ Dimensionen nahezu vollständig beschrieben werden können. Jede dieser $K$ Dimensionen beschreibt ein für die Kategorie (z.B. Gesichtsbilder) relevantes Merkmal. Im Fall der Gesichtserkennung werden die relevanten Merkmale auch als Eigenfaces bezeichnet. Jedes Eigenface kann als Bild dargestellt werden, welches ein bestimmtes Gesichtsmerkmal besonders hervorhebt. Jedes individuelle Bild der Kategorie (d.h. jedes Gesicht) kann dann als Linearkombination der $K$ relevanten Merkmale (der $K$ Eigenfaces) beschrieben werden.\\

Das Problem besteht nun zunächst darin, aus einer Menge von Bildern der gleichen Kategorie die relevanten Merkmale zu finden. Dieses Problem wird durch die Principal Component Analysis (PCA) gelöst. Die PCA, findet in einer Menge von Bildern der gleichen Kategorie die Hauptachsen, also die Richtungen im $N^2$-dimensionalen Raum, entlang derer die Varianz zwischen den gegebenen Bildern am stärksten ist. Der $N^2$-dimensionale Pixelraum wird dann in einen Raum, der durch die gefundenen Hauptachsen aufgespannt wird, transformiert. In diesem in der Anzahl der Dimensionen stark reduzierten Raum wird dann die Bilderkennung durchgeführt. Der hier skizzierte Ansatz der Eigenfaces für die Gesichtserkennung wurde erstmalig in [M. Turk, A. Pentland; Eigenfaces for Recognition](http://www.cs.ucsb.edu/%7Emturk/Papers/jcn.pdf) beschrieben.

### Genereller Ablauf

Die Gesichtserkennung besteht aus 2 Phasen. In der Trainingsphase werden die Gesichtsbilder der zu erkennenden Personen eingelesen und für diese mit der PCA der Eigenface-Raum berechnet. In der Erkennungsphase wird ein neu aufgenommenes Bild in den Eigenface-Raum transformiert und dort dem naheliegendsten Bild aus der Trainingsmenge zugeordnet.

#### Trainingsphase

1. Lese Gesichtsbilder der Personen, die erkannt werden sollen ein. Die Menge dieser Bilder definiert das Trainingsset
2. Berechne mit der PCA den Eigenface-Raum. Dabei werden nur die K Dimensionen, welche zu den Eigenvektoren mit den größten Eigenwerten gehören ausgewählt. Die zu den K Dimensionen (Eigenvektoren) gehörenden Bilder sind die Eigenfaces.
3. Transformiere jedes Bild der Trainingsmenge in den Eigenface-Raum und erhalte so die entsprechende Repräsentation des Bildes als Punkt im Eigenface-Raum.

#### Erkennungsphase

1. Transformiere das zu erkennende Bild in den Eigenface-Raum und berechne dort die Koordinaten des Bildes hinsichtlich aller K-Dimensionen (Eigenfaces)
2. Bestimme ob das zu erkennende Bild überhaupt eine Gesicht darstellt
3. Bestimme ob das Gesicht zu einer bekannten Person, deren Bild in der Trainingsmenge enthalten ist, gehört.

#### Update (optional)

Füge das erkannte Bild zur Menge der Trainingsbilder hinzu und führe die Schritte der Trainingsphase durch.

### Bestimmung der Eigenfaces
<a id='theoryEig'></a>
Es werden zunächst $M$ Gesichtsbilder der zu erkennenden Personen eingelesen (von jeder zu erkennenden Personen möglichst mehrere Bilder). Es wird davon ausgegangen, dass jedes der Bilder $C$ Pixel breit und $R$ Pixel hoch ist. Das Bild kann dann als $R \times C$ Matrix dargestellt werden. Im Fall eines Graustufenbildes repräsentieren die Pixelwerte den entsprechenden Grauwert. Nach dem Einlesen werden die Bildmatrizen als eindimensionale Vektoren dargestellt. Für diese Umformung werden die Zeilen jeder Matrix von oben nach unten ausgelesen und hintereinander gereiht. Jedes Bild wird dann durch einen Vektor der Länge $Z=R \cdot C$ repräsentiert und kann als Punkt im Z-dimensionalen Raum dargestellt werden. Die M Bildvektoren werden im folgenden mit $$\Gamma _1, \Gamma_2, \ldots, \Gamma_M$$ bezeichnet.

Im nächsten Schritt wird das Durchschnittsbild berechnet
$$
\overline{\Gamma}=\frac{1}{M}\sum_{i=1}^{M}{\Gamma_{i}}
$$

Dieses Durchschnittsbild wird von allen Bildern $\Gamma_i$ abgezogen. Die Menge der so gewonnenen Bildrepräsentationen
$$
\Phi_i=\Gamma_i - \overline{\Gamma}
$$

ist dann mittelwertsfrei. Die Menge  $\Phi_1, \Phi_2, \ldots, \Phi_M$ wird dann einer Principal Component Analysis (PCA) (siehe auch [J. Maucher; Feature Selection and Extraction](https://docs.google.com/document/d/13cc9RGeIvsV5JsC-MsymJ6aaJs_ZsUKEwqaOOO8IL8Q/edit?usp=sharing)) unterzogen. Hierzu werden die mittelwertfreien Bildrepräsentationen $\Phi_i$ in die Spalten einer Matrix geschrieben. Diese Matrix wird im Folgenden mit $X$ bezeichnet. Unter der Annahme, dass die $\Phi_i$ bereits als Spaltenvektoren vorliegen, ist die Matrix $X$ definiert als:

$$
X=\left[ \Phi_1, \Phi_2, \ldots, \Phi_M \right].
$$

Die entsprechende Kovarianzmatrix ist dann
$$
CV=X \cdot X^T.
$$

Für die PCA müssten als nächstes eigentlich die Eigenvektoren und Eigenwerte der Kovarianzmatrix $CV$ berechnet werden. Für den vorliegenden Fall kann allerdings die hierfür notwendige Berechnung aus Komplexitätsgründen nicht realisiert werden. Man beachte dass die Matrix $CV$  $Z$ Spalten und $Z$ Zeilen enthält ($Z$ ist die Anzahl der Pixel in einem Bild) und für diese $Z$ Eigenvektoren und Eigenwerte existieren. Wie in [M. Turk, A. Pentland; Eigenfaces for Recognition](http://www.cs.ucsb.edu/%7Emturk/Papers/jcn.pdf) beschrieben, existieren im Fall, dass die Anzahl der Bilder $M$ wesentlich kleiner als die Anzahl der Pixel $Z$ ist, nur $M-1$ __relevante Eigenvektoren__, die Eigenwerte aller anderen Eigenvektoren liegen nahe bei Null. Der in [M. Turk, A. Pentland; Eigenfaces for Recognition](http://www.cs.ucsb.edu/%7Emturk/Papers/jcn.pdf) beschriebene Ansatz geht nun von der $M \times M$ Matrix

$$
X^T \cdot X
$$

aus, für welche die Eigenvektoren und Eigenwerte für eine moderate Bildanzahl $M$ gut berechnet werden können. Per Definition gilt für die Eigenvektoren $\mathbf{v}_i$ und Eigenwerte $\mu_i$ dieser Matrix:

$$
X^T \cdot X \cdot \mathbf{v}_i = \mu_i \mathbf{v}_i .
$$

Werden beide Seiten dieser Matrix linksseitig mit der Matrix $X$ multipliziert,

$$
X \cdot X^T \cdot X \cdot \mathbf{v}_i = \mu_i X \mathbf{v}_i,
$$
dann ist daraus zu erkennen, dass die $M$ Vektoren
$$
\mathbf{u}_i=X \mathbf{v}_i
$$

die Eigenvektoren der Matrix $$CV=X \cdot X^T$$ sind. D.h. es können zunächst die M Eigenvektoren der relativ kleinen Matrix $X^T \cdot X$ bestimmt und aus diesen durch eine einfache Multiplikation mit der Matrix $X$ die relevanten Eigenvektoren der Matrix $CV$ berechnet werden. Da die Matrix $X$ die $M$ Bildrepräsentationen  $\Phi_i$ als Spalten enthält, können die gesuchten Eigenvektoren auch als Linearkombination der $M$ Bilder der Trainingsmenge beschrieben werden:

$$
\mathbf{u}_i=\sum_{k=1}^{M}{v_{i,k}\Phi_k}
$$

wobei mit $v_{i,k}$ die $k.$te Komponente des Vektors $\mathbf{v}_i$ bezeichnet wird. Die Eigenvektoren $\mathbf{u}_i$ werden auch Eigenfaces genannt. Per Definition sind die Eigenvektoren paarweise orthogonal. Jeder Eigenvektor ist ein Spaltenvektor mit $Z$ (=Anzahl der Pixel) Komponenten.

Die $M$ Eigenvektoren werden dann entsprechend der Größe der zugehörigen Eigenwerte $\mu_i$ geordnet. Für die weiteren Schritte kann zum Zwecke einer weiteren Komplexitätsreduktion eine Untermenge der $K$ relevantesten Eigenvektoren benutzt werden (also der $K$ Eigenvektoren mit den höchsten Eigenwerten). Beispielsweise ist in [M. Turk, A. Pentland; Eigenfaces for Recognition](http://www.cs.ucsb.edu/%7Emturk/Papers/jcn.pdf) für die Erkennung von $M=16$ Personen und eine Auflösung von $256 \times 256$ Pixel meist $K=7$ Eigenvektoren für eine gute Erkennung ausgereicht.

### Gesichtserkennung im Eigenspace
<a id='theoryRec'></a>

Die $K$ ausgewählten Eigenvektoren $\mathbf{u}_1,\mathbf{u}_2,\ldots \mathbf{u}_K$ spannen einen $K-$dimensionalen Raum, den sogenannten Eigenspace auf. Die $K$ Vektoren repräsentieren die $K$ Merkmale hinsichtlich derer die Bilder der Trainingsdatenmenge am stärksten variieren.

Für die Bilderkennung wird jetzt jedes Bild, also sowohl die Bilder aus der Trainingsmenge als auch die zu erkennenden Bilder, in den Eigenspace transformiert. Jedes Bild stellt einen Punkt im Eigenspace dar. Für die Erkennung kann einfach die Distanz des zu erkennenden Bildes zu allen Bildern der Trainingsmenge berechnet werden. Das zu erkennende Bild wird der Person (Bildklasse) zugeordnet, deren Punkt im Eigenspace dem Punkt des zu erkennenden Bildes am nächsten liegt.

Die $K$ Komponenten eines Trainingsbildes werden berechnet, indem das Bild auf den jeweiligen Eigenvektor projiziert wird. Demnach ist die $k.$te Komponente des $i.$ten Trainingsbildes  $\Phi_i$:
$$
\omega_{k,i}=\mathbf{u}_k^T \Phi_i
$$

Der dem Bild $\Phi_i$ entsprechende Punkt im Eigenspace ist dann
$$
\mathbf{w}_i=[\omega_{1,i},\omega_{2,i},\ldots,\omega_{K,i}].
$$

Wird mit $\Gamma$ das zu erkennende Bild und mit $\Phi=\Gamma - \overline{\Gamma}$ die um den Mittelwert der Trainingsbilder subtrahierte Version des Bildes bezeichnet, dann sind
$$
\omega_{k}=\mathbf{u}_k^T \Phi
$$

die Koordinaten der Projektion von $\Phi$ in den Eigenspace und der dieses Bild repräsentierende Punkt
$$
\mathbf{w}=[\omega_{1},\omega_{2},\ldots,\omega_{K}].
$$

Das zu erkennende Bild wird dann dem Trainingsbild $\Phi_j$ zugeordnet, für welches gilt:
$$
j=argmin_{i} \left\{ d(\mathbf{w},\mathbf{w}_i) \right\}
$$

wobei mit $d(\mathbf{w},\mathbf{w}_i)$ die euklidische Distanz zwischen den Projektionen von $\Phi$ und $\Phi_i$ bezeichnet wird.

__Optional:__ Falls $\Phi_i$ nicht das einzige Bild einer Person in der Trainingsmenge ist, sondern für die entsprechende Person mehrere Trainingsbilder vorliegen, wird in der Distanzberechnung nicht $\Phi_i$, sondern der Mittelwert über alle zu dieser Person gehörenden Bilder eingesetzt:

$$
\overline{\Phi}=\frac{1}{|W|}\sum_{w \in W}^{}{\Phi_w} .
$$

Dabei bezeichnet $W$ die Menge aller der Indizees $w$, für die die $\Phi_w$ zur gleichen Person gehören. Im Praktikumsversuch muss diese Option nicht implementiert werden. Die im folgenden Abschnitt beschriebene Versuchsdurchführung bezieht sich auf den Fall, dass nur die Distanz zu Einzelbildern berechnet wird und das nächstliegende Bild ausgegeben wird.

Für die Mindestdistanz
$$
\epsilon =\min_{i} \left\{ d(\Phi,\Phi_i) \right\}
$$

wird in der Regel eine Schwelle $T$ definiert. Wenn $\epsilon > T$ ist, also eine relativ große Distanz zwischen dem zu erkennenden Bild und dem nächstliegenden Bild aus der Trainingsmenge besteht, wird davon ausgegangen, dass es sich um ein unbekanntes Gesicht handelt. Optional könnte dieses unbekannte Gesicht in die Trainingsmenge aufgenommen werden.

Schließlich muss noch der Fall behandelt werden, dass das eingelesene Bild kein Gesicht darstellt. Aufgrund der starken Projektion vom ursprünglichen Bildraum in den Eigenspace kann dieser Fall nicht durch eine Schwelle auf den Fehler $\epsilon$ erkannt werden. Es kann durchaus sein, dass ein Nicht-Gesichtsbild in die Umgebung eines Gesichtsbild im Eigenspace projiziert wird. Ein Nicht-Gesichtsbild wird aber eine relativ große Distanz $d(\Phi,\Phi_f)$ zwischen

$$
\Phi=\Gamma - \overline{\Gamma}
$$
und der Repräsentation im Eigenspace
$$
\Phi_f=\sum_{k=1}^{K}{\omega_k}\mathbf{u}_k
$$
aufweisen. Durch die Definition einer weiteren Schwelle $S$ auf $d(\Phi,\Phi_f)$ kann also erkannt werden, ob es sich überhaupt um ein Gesicht handelt. Im Versuch ist davon auszugehen, dass nur Gesichtsbilder verwendet werden, d.h. es muss nur der Test gegen die Schwelle $\epsilon$ implementiert werden.


 ## Vor dem Versuch zu klärende Fragen

 * Was sind Eigenvektoren und Eigenwerte?

Eigenvektoren sind die dimensionsreduzierten Vektoren die ein Bild beschreiben, Eigenwerte sind die jeweiligen Werte
dieser Vektoren.

* Was versteht man unter Eigenfaces?

Darstellung eines Bildes im K-Dimensionalen Raum.

* Die PCA ist u.a. im entsprechenden Kapitel des Dokuments [J. Maucher; Feature Selection and Extraction](https://docs.google.com/document/d/13cc9RGeIvsV5JsC-MsymJ6aaJs_ZsUKEwqaOOO8IL8Q/edit?usp=sharing)) beschrieben. Wie kann mit der PCA eine Dimensionalitätsreduktion durchgeführt werden?

Bei der PCA werden die Dimensionen mit dem niedrigsten Informationsgehalt berechnet und dann aus dem Vektor entfernt.
Ein hoher Informationsgehalt ist dabei eine hohe Varianz.

* Wie werden mit dem Python Modul \emph{Image} Bilder in ein Python-Programm geladen?

image.open("path")

# Versuchsdurchführung

## Einlesen der Gesichtsbilder in Numpy Arrays
Laden Sie vom Skripteserver das Verzeichnis _Gesichtsbilder_ auf Ihren Rechner. Darin enthalten sind

* das Unterverzeichnis _training_, welches je 3 Bilder jedes Studenten enthält,
* das Unterverzeichnis _test_, welches die nicht in _training_ enthaltenen Bilder enthält. Ein Bild von jedem Studenten.

Mit der unten gegebenen Funktion _parseDirectory(directoryName,extension)_ wird eine Liste aller Dateinamen des Typs _extension_ im Verzeichnis _directoryName_ angelegt.

__Aufgabe:__

1. Legen Sie mit dieser Funktion eine Liste mit allen Dateinamen des Typs _extension='.png'_ im Verzeichnis _training_ (enthält die Trainingsbilder) an.
2. Implementieren Sie eine Funktion _readImageToNumpyData(imageList)_, der eine Liste aller Dateinamen der Trainingsbilder übergeben wird. Die Funktion gibt ein Numpy-Array zurück. Jede Zeile dieses Arrays enthält ein _.png_-Bild in serialisierter Form. Hierzu ist jedes einzelne Bild zunächst mit der Funktion [_matplotlib.image.imread(filename)_](http://matplotlib.org/1.3.0/users/image_tutorial.html) in ein zweidimensionales Numpy Array _img_ zu lesen. Durch den Aufruf von _img.shape=(1,-1)_ wird das zweidimensionale Numpy Array zu einem eindimensionalen Array serialisiert. Danach muss eine Normierung aller Werte in den Bereich zwischen 0 und 1 durchgeführt werden. Hierzu müssen alle Pixelwerte eines Bildes durch den im jeweiligen Bild vorkommenden Maximalwert geteilt werden.

In [ ]:
# % matplotlib inline
from os.path import isdir, join, normpath
from os import listdir
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as mplimg

In [ ]:
from pandas import DataFrame
from scipy.spatial.distance import euclidean
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report


def parseDirectory(directoryName, extension):
    '''This method returns a list of all filenames in the Directory directoryName.
    For each file the complete absolute path is given in a normalized manner (with
    double backslashes). Moreover only files with the specified extension are returned in
    the list.
    '''
    if not isdir(directoryName): return
    imagefilenameslist = sorted([
        normpath(join(directoryName, fname))
        for fname in listdir(directoryName)
        if fname.lower().endswith('.' + extension)
    ])
    return imagefilenameslist

Funktion, welche alle gegebenen Dateien als Bild laedt, und in ein *numpy*-Array schreibt.
Die Bilder werden gleichzeitig in ein eindimensionalen Vektor "geflacht" und auf einen Wertebereich von 0 bis 1 gebracht.

In [ ]:
def read_images_to_numpy_data(image_files):
    images = np.array([mplimg.imread(join(".", image_path)) for image_path in image_files])
    images_flattened = np.reshape(images, (images.shape[0], -1))
    images_normed = images_flattened / np.max(images_flattened)
    return images_normed

Funktion, die alle Bilder in einem gegebenen Order mithilfe der vorherigen Funktion laedt.

In [ ]:
def get_images_from_directory(dir, ext="png"):
    relative_dir = join(".", dir)
    image_files = parseDirectory(relative_dir, ext)
    images_raw = read_images_to_numpy_data(image_files)
    return images_raw

Nun wird die vorherige Funktion mit dem Order mit den Trainingsbildern ausgefuehrt.

In [ ]:
images_raw = get_images_from_directory("FaceRecogBilder/training")
images_raw.shape

Die Form des Arrays zeigt, dass es 63 Zeilenvektoren in der Matrix gibt, also 63 eindimensionale Vektoren, die
jeweils ein Bild repraesentieren.

In [ ]:
images_raw[:5]

Man kann gut erkennen, dass es sich hier um ein *numpy*-Array handelt, und die Werte in einem Bereich zwischen 0 und 1 sind.
Um sicher zu gehen, dass es wirklich nur der Wertebereich zwischen 0 und 1 ist, werden nun die maximalen und minimalen
Werte des Arrays ausgegeben.

In [ ]:
print(f"Min value in array: {np.min(images_raw)}, max value in array: {np.max(images_raw)}")

Wie gut zu erkennen ist, hat das Normieren erfolgreich funktioniert.

## Berechnung des Durchschnittbildes

__Aufgaben:__

1. Die von der Funktion _convertImgListToNumpyData(imgList)_ zurückgelieferte Matrix enthält in ihren Zeilen alle Trainingsbilder. Aus diesen Trainingsbildern ist nach der Gleichung für [$\overline{\Gamma}$](#theoryEig) das Durchschnittsbild zu berechnen, z.B. durch Anwendung der Numpy-Funktion _average_. Das Durchschnittsbild ist von allen Bildern abzuziehen (Gleichung [$\Phi_i$](#theoryEig)). Das daraus resultierende Numpy-Array enthält die mittelwertfreien Repräsentationen der Trainingsbilder und wird im Folgenden mit _NormedArrayOfFaces_ bezeichnet.

2. Zeigen Sie das Durchschnittsbild mithilfe der [matplotlib.pyplot_-Funktion _imshow()](http://matplotlib.org/1.3.0/users/image_tutorial.html) an. Hierzu muss das eindimensionale Numpy Array, welches das Durchschnittsbild enthält, in ein zweidimensionales Array der ursprünglichen Bildgröße umgewandelt werden (Numpy Funktionen _shape()_ oder _reshape()_)

__Wichtiger Hinweis:__ Das Numpy-Array __NormedArrayOfFaces__ ist die Transpornierte $X^T$ der Matrix $X$ aus Gleichung [$X$](#theoryEig).

Funktion zur Ermittlung des **mean images** eines Image-Arrays.

In [ ]:
def get_mean_image(images):
    return np.mean(images, axis=0)

Funktion, die ein gegebenes *mean-image* von einem Image-Array abzieht.

In [ ]:
def normalize_images(images, mean_image):
    return images - mean_image

Funktion, die ein Image-Array mittelwertsfrei macht, indem das Mittel-Bild abgezogen wird.
Zusaetzlich wird das Array als vorbereitenden Schritt transponiert, sodass die Bilder als Spaltenvektoren vorliegen.

In [ ]:
def get_normed_images(images, mean_image=None):
    if mean_image is None:
        mean_image = get_mean_image(images)
    return np.transpose(normalize_images(images, mean_image))

Um die Transformationen zu ueberpruefen, wird nun die Form des entstandenen Bild-Arrays ausgegeben.

In [ ]:
images_normed = get_normed_images(images_raw)
images_normed.shape


Wie man gut erkennen kann, liegen die Bilder als Spaltenvektoren vor. Es gibt 63 Spaltenvektoren, welche jeweils
ein Trainings-Bild darstellen.

Jetzt wird das Mittelwertbild der Trainins-Bilder ermittelt.

In [ ]:
mean_face = get_mean_image(images_raw)

Um ein besseres Verstaendnis eines solchen Mittelwertbildes zu bekommen, wird dieses nun ausgegeben.
Dabei ist darauf zu achten, dass der flache eindimensionale Vektor wieder in die richtige Form, also eine Matrix in der
Form des urspruenglichen Bildes, gebracht wird.

In [ ]:
def show_image(flattened_image, width, height):
    image_reshaped = np.reshape(flattened_image, (height, width))
    plt.imshow(image_reshaped, cmap="gray")
    plt.show()


show_image(get_mean_image(images_raw), 150, 220)

Man kann deutliche Gesichtszuege erkennen. Da die Trainingsbildermenge relativ gering ist, kann man auch markante
Merkmale, wie zum Beispiel Brillen, gut erkennen.

Um einen Eindruck davon zu bekommen, was ein mittelwertfreies Bild ist, wird nun das erste mittelwertfreie Bild des
Datensatzes ausgegeben.

In [ ]:
show_image(images_normed[:, 0], 150, 220)

Im Vergleich zu dem untransformierten Bild, kann man hier deutliche helle Flecken erkennen. Wie man im Mittelwertbild
erkennen kann, kommt dies dadurch, dass im oberen Drittel des Bildes meistens die Haare sind, welche in einer Graustufen-
Repraesentation fast immer als dunklere Pixelwerte dargestellt werden, sofern sie nicht blond sind.
Wird nun dieses Mittelwertsbild von den Trainingsbildern abgezogen, kommt es zu den helleren Flecken im oberen Drittel.
Somit sind nur noch die relevanten Pixel und Merkmale, die sich am meisten vom Mittel der Trainingsdaten unterscheidet,
im Bild vorhanden.

## Berechnung der Eigenfaces

__Aufgaben:__

1. Implementieren Sie die Funktion _calculateEigenfaces(adjfaces,width,height)_. Dieser Funktion werden die normierten Bilder _NormedArrayOfFaces_ zusammen mit der Bildbreite und -höhe übergeben. Zurück liefert die Funktion ein Numpy-Array, dessen Zeilen die berechneten normierten Eigenfaces sind. Die Berechnung der Eigenfaces ist im Theorieteil Abschnitt [Bestimmung der Eigenfaces](#theoryEig) beschrieben. Für die Python-Implementierung können Sie folgende Hinweise berücksichtigen:
    * Berechnung der transponierten eines Numpy-Arrays $A$ mit der Numpy-Methode _transpose()_
    * Matrixmultiplikation zweier Numpy-Arrays $A$ und $B$ mit der Numpy-Funktion _dot()_
    * Berechnung der Eigenvektoren und Eigenvalues eines Numpy Arrays $A$ mit der Numpy-Funktion _linalg.eigh()_
    * Sortierung von Numpy-Arrays mit den Numpy-Funktionen _sort()_ und _argsort()_.
2. Aus dem von der Funktion _calculateEigenfaces(adjfaces,width,height)_ zurück gelieferten Array von Eigenfaces sind die $K$ relevantesten auszuwählen. Dieses reduzierte Array wird im Folgenden mit _Usub_ bezeichnet. Im Versuch kann $K=6$ eingestellt werden.
3. Zeigen Sie die $K=6$ wichtigsten Eigenfaces als Bilder mit der [matplotlib.pyplot_-Funktion _imshow()](http://matplotlib.org/1.3.0/users/image_tutorial.html) an.

Nun wird die Funktion fuer das Berechnen der Eigenfaces definiert.
Hier wird als Input lediglich das normierte und zentrierte Array mit den Trainingsbildern benoetigt. Die Breite und
Hoehe der Bilder mit anzugeben, wie es in der Aufgabe heisst, konnte nicht nachvollzogen werden, da sie in keine Schritt
benoetigt werden.

Um die **Singular-Value-Decomposition** zu vereinfachen, kann wird sich der *linal*-Bibliothek von *numpy* bedient.
Sie liefert die Eigenwerte und -vektoren zurueck, welche dann weiter verwendet werden koennen. Fuer diesen Versuch
sind nur die Eigenvektoren wichtig, da die *svd*-Funktion bereits nach Eigenwerten sortierte Eigenvektoren ausgibt.
Um zu verhindern, dass die komplette Covarianz-Matrix berechnet wird, wird die Parameters *fullmatrices=0* mitgegeben.
Dieser sorgt dafuer, dass nur die "Economy-Matrix* berechnet wird, also die $CV=X^T \cdot X$ Matrix.

Wie man an der Form des entstehenden Arrays erkenne kann, wurden 63 neue Spaltenvektoren mit jeweils der Laenge 33000
erzeugt. Diese Spaltenvektoren repraesentieren die sogenannten Eigenfaces, mit denen Gesichter abstrahiert werden koennen.

In [ ]:
def calculate_eigenfaces(normed_images):
    U, _, _ = np.linalg.svd(normed_images, full_matrices=0)
    return U


eigenfaces = calculate_eigenfaces(images_normed)
eigenfaces.shape

Da eine weitere Komplexitaetsreduktion gewuenscht ist, koennen mit der folgenden Funktion, die x informationsreichsten
Eigenfaces aus dem Array aller Eigenfaces extrahiert werden. Der Default-Wert liegt bei einem experimentell ermittelten
Wert von 7 Eigenfaces.

In [ ]:
def get_most_relevant_eigenfaces(eigenfaces, k=7):
    return eigenfaces[:, :k]


eigenface_space = get_most_relevant_eigenfaces(eigenfaces)
eigenface_space.shape

Nun werden diese 7 ermittelten Eigenfaces ausgegeben.

In [ ]:
for i, image in enumerate(np.transpose(eigenface_space)):
    print(f"Eigenface #{i + 1}")
    show_image(image, 150, 220)

Wie man gut erkennen kann, sind diese Eigenfaces sehr abstrakte Darstellungen von menschlichen Gesichtern, oder besser
gesagt, die wichtigsten Merkmale der Trainings-Gesichter. Mit absteigendem Informationsgehalt werden die Features immer
markanter. Das erste Eigenface beinhaltet praktisch nur die sehr grobe Form eines Gesichts sowie die Haare. Im 7. Eigenface
kann man schon spezifischere Merkmale, wie Brillen oder besondere Mundformen erkennen.
Dies bedeutet, dass mit absteigendem Informationsgehalt Eigenfaces immer spezifischer werden, und dadurch nur spezifischen
Gesichtern bei der Darstellung helfen, waehrend Eigenfaces mit hoeherem Informationsgehalt zu einem gewissen Anteil in
die Rekonstruktion jedes Gesichts einfliessen.

## Transformation der normierten Trainingsbilder in den Eigenface Raum

__Aufgabe:__

Die im vorigen Schritt angelegten $K$ relevantesten Eigenfaces spannen den sogenannten _Eigenface-Raum_ auf. Für jedes der normierten Trainingsbilder, also für jede Zeile aus _NormedArrayOfFaces_, sind die Koordinaten im Eigenface-Raum entsprechend der Gleichung für [$\omega_{k,i}$](#theoryRec) definierten Transformation zu berechnen.

Nun soll der obrige Algorithmus auf die Trainingsdaten angewandt werden.

Zuerst wird der komplette Traininsdatensatz geladen und normiert.

In [ ]:
train_image_dir = "FaceRecogBilder/training"
train_image_files = parseDirectory(train_image_dir, "png")
train_images_raw = get_images_from_directory(train_image_dir)
train_images_normed = get_normed_images(train_images_raw)
train_images_normed.shape

Nun wird eine Funktion definiert,  welche ein gegebenes, normiertes, Bild in Koordinaten in einem gegebenen
Eigenface-Space transformiert. Diese Koordinaten sind quasi der *Fingerabdruck* dieses Bildes im Eigenface-Space und
das eigentliche Bild kann mit diesen Koordinaten und dem Eigenface-Space annaehernd gekonstruiert werden.

In [ ]:
def get_eigenspace_coordinates(normed_images, eigenface_space):
    return eigenface_space.T @ normed_images

Nun werden alle Bilder aus dem Trainingsdatensatz in den Eigenspace transformiert.

In [ ]:
# train_eigenspace = get_eigenspace_coordinates(train_images_normed, eigenface_space)
train_eigenspace = np.apply_along_axis(get_eigenspace_coordinates, 0, train_images_normed, eigenface_space)
train_eigenspace.shape

Wie gut zu erkennen ist, ist jedes Bild jetzt nicht mehr mit einem eindimensionalen Vektor der Laenge 33000 dargestellt,
sondern lediglich ein eindimensionaler Vektor der Laenge 7, also der Laenge des Eigenface-Space.

Dies zeigt, wie viel Komplexitaet tatsaechlich reduziert werden kann mit diesem Algorithmus.

In [ ]:
train_eigenspace[:, 0]

Wie man an der Ausgabe erkennen kann, sind die Koordinaten an sich nich sonderlich aussagekraeftig.
Nun soll gezeigt werden, wie mithilfe des Eigenface-Space das originale Bild nur mit den gegebenen Eigenspace-Koordinaten
rekonstruiert werden kann.
Hierfuer wird eine Funktion implementiert, welche die Eigenspace-Koordinaten, den Eigenface-Space, sowie das
Mittelwertsbild vom Anfang nimmt, und daraus durch eine Matrixmultiplikations und die Addition des Mittelwertbildes das
Ursprungsbild wiederherstellen kann.

In [ ]:
def reconstruct_image(eigenspace_coordinates, mean_image, eigenspace):
    return mean_image + eigenspace @ eigenspace_coordinates

Dies wird nun mit dem ersten Bild der Trainingsdaten ausprobiert.

In [ ]:
original_image = train_images_raw[0]
show_image(original_image, 150, 220)

In [ ]:
reconstructed_image = reconstruct_image(train_eigenspace[:, 0],
                                        get_mean_image(train_images_raw),
                                        eigenface_space)
show_image(reconstructed_image, 150, 220)

Wie gut erkennbar ist, sind die wichtigsten Merkmale des urspruenglichen Bildes rekonstruiert worden.
Da hierfuer lediglich die ersten 7 Eigenfaces verwendet wurden, und dies nicht ausreicht, um das Bild gut zu
rekonstruieren, wird dies nun noch einmal mit verschiedenen Anzahlen von Eigenfaces ausgegeben.

In [ ]:
k_values = [7, 10, 15, 20, 30, 50, 63]
for k_value in k_values:
    eigenface_space_temp = get_most_relevant_eigenfaces(eigenfaces, k_value)
    train_eigenspace_temp = np.apply_along_axis(get_eigenspace_coordinates, 0, train_images_normed,
                                                eigenface_space_temp)

    reconstructed_image = reconstruct_image(train_eigenspace_temp[:, 0],
                                            get_mean_image(train_images_raw),
                                            eigenface_space_temp)
    print(f"Eigenspace-Length: {k_value}")
    show_image(reconstructed_image, 150, 220)

Wie man gut erkennen kann, werden die Rekonstruktionen besser, je mehr Dimensionen der Eigenface-Space hat.
Bei einem niedrigen K ist es noch schwierig auf das Ursprungsbild zu schliessen, spaetestens ab einem K von 30 ist es
aber eindeutig, welches Bild rekonstruiert wurde.
Der Unterschied zwischen k=50 und k=63 ist so minimal, das deutlich wird, wie wenig Informationen die hohen Dimensionen
des Eigenface-Space enthalten, und dort viel Speicherplatz und Rechenaufwand gespart werden kann ohne wesentlich Informationen
zu verlieren.
Vor allem hinsichtlicher eines Komprimierungsalgorithmus hat sich diese Theorie als sehr hilfreich erwiesen.

## Erkennung

__Aufgaben:__

1. Wählen Sie ein Bild aus dem Verzeichnis _test_ aus. Das ausgewählte zu erkennende Bild ist als Numpy-Array darzustellen. Eine Normierung der Pixelwerte in den Bereich zwischen 0 und 1 ist durchzuführen (wie bereits oben beschrieben). Schließlich muss auch von diesem Bild das Durchschnittsbild aller Trainingsbilder abgezogen werden. Diese Prozessschritte entsprechen der oben beschriebenen Vorverarbeitung der Trainingsbilder. Das resultierende normierte und mittelwertfreie Bild wird im Folgenden mit _NormedTestFace_ bezeichnet.
3. Danach sind die Koordinaten des _NormedTestFace_ im Eigenface-Raum nach Gleichung [$\omega_{k}$](#theoryRec) zu berechnen und das in diesem Raum nächstliegende Trainingsbild zu bestimmen.


Um den Algorithmus zu testen, wird er nun auf unbekannte Testdaten angewandt.

Zuerst werden diese Testdaten geladen und normiert.
Wichtig hierbei ist, dass nicht das Mittelwertsbild der Testdaten, sondern das der Trainingsdaten verwendet wird.

In [ ]:
test_image_dir = "FaceRecogBilder/test"
test_image_files = parseDirectory(test_image_dir, "png")
test_images_raw = get_images_from_directory(test_image_dir)
test_images_normed = get_normed_images(test_images_raw, get_mean_image(train_images_raw))
test_images_normed.shape

Die geladenen Test-Bilder werden nun in den generierten Eigenface-Space transformiert.

In [ ]:
test_eigenspace = np.apply_along_axis(get_eigenspace_coordinates, 0, test_images_normed, eigenface_space)
test_eigenspace.shape

Fuer den initialen Test wird nun das erste Bild der Testdaten genommen.

In [ ]:
test_item_index = 0
test_item = test_eigenspace[:, test_item_index]
test_item

Um die Aehnlichkeit von Gesichtern zu berechnen, wird nun eine Distanz-Funktion implenetiert.
Diese berechnet die euklidische Distanz zwischen zwei Punkten im Eigenspace. Je naeher sie aneinander sind, desto
aehnlicher ist das dazugehoerige Bild, respektive Gesicht.

In [ ]:
def get_single_distance(i_1, i_2, ):
    return euclidean(i_1, i_2)

Mit der folgenden Funktion kann die Distanz-Funktion auf einen gesamten Datensatz angewandt werden.

In [ ]:
def get_distances(image, comp_images):
    return np.apply_along_axis(get_single_distance, 0, comp_images, image)

Nun werden die Distanzen des Testbildes zu allen Trainingsbildern ermitteln.
Da sich in den Trainingsbildern jeweils zu den Testbildern passende Gesichter befinden, kann gut ermittelt werden,
ober der Algorithmus gute Ergebnisse produziert.

In [ ]:
distances = get_distances(test_item, train_eigenspace)
distances

Um zu ermitteln, zu welchem Bild die jeweilige Distanz gehoert, wird nun eine Funktion implementiert, welche den Index
des jeweiligen Bildes mit der kleinsten Distanz ausgibt.

In [ ]:
def get_most_similar_item_index(item_array):
    most_similar_item_index = np.where(item_array == np.min(item_array))
    return most_similar_item_index[0][0]

In [ ]:
most_similar_item_index = get_most_similar_item_index(distances)
most_similar_item_index

Wenn man diesen Index nun von den initialen Bilddateien nimmt, erhaelt man den Dateinamen des Bildes, welches dem Testbild
am aehnlichsten ist.

In [ ]:
most_similar_item_name = train_image_files[most_similar_item_index]
most_similar_item_name


Um zu ueberpruefen, ob es sich wirklch um das selbe Gesicht handelt, werden nun das initiale Testbild, sowie das
ermittelte aehnlichste Bild angezeigt.

In [ ]:
show_image(test_images_raw[test_item_index], 150, 220)

In [ ]:
show_image(train_images_raw[most_similar_item_index], 150, 220)

Wie gut zu erkennen ist, handelt es sich um die selbe Person. Dies bedeutet, dass der Algorithmus ein richtiges Ergebnis
produziert hat.
Um dies zu validieren, wird in der folgenden Aufgabe die Performance der Erkennung evaluiert.

__Aufgaben:__
1. Führen Sie die implementierte Gesichtserkennung für alle Bilder im Verzeichnis _test_ aus. Zeigen Sie jeweils das Testbild, das zugehörige erkannte Bild und die Distanz zwischen beiden Bildern an.
2. Bestimmen Sie für die Werte $K=5,K=10$ und $K=15$ ($K$ ist die Anzahl der verwendeten Eigenfaces) die Rate falsch erkannter Bilder. 


Um die Performance des Algorithmus zu ermitteln, werden zunaechst die Distanzen aller Testbilder zu den Trainingsbildern
berechnet.

In [ ]:
item_distances = np.apply_along_axis(get_distances, 0, test_eigenspace, train_eigenspace)
item_distances.shape

Ausgehend davon, wird nun ermittelt, um welche Bilder es sich jeweils handelt.

In [ ]:
# most_similar_item_files = [train_image_files[get_most_similar_item_index(item)] for item in item_distances]
most_similar_item_files = [train_image_files[get_most_similar_item_index(item_distances[:, i])] for i in
                           range(item_distances.shape[1])]
most_similar_item_files[:5]

Um einen Einblick in die Evaluationsperformance des Algorithmus zu bekommen, werden jeweils die Eingabe-Testbilder, passende
Ausgabe-Trainingsbild, sowie die Distanz zwischen den beiden ausgegeben.

In [ ]:
def print_image_pair(test_image, image_match, distance):
    print("Test image:")
    if isinstance(test_image, str):
        test_image = mplimg.imread(test_image)
    show_image(test_image, 150, 220)

    print("Matching Image with lowest distance:")
    if isinstance(image_match, str):
        image_match = mplimg.imread(image_match)
    show_image(image_match, 150, 220)

    print(f"Distance between images: {distance}")
    print("------------------------------")

In [ ]:
data_triplets_6 = [(test_image_files[i], most_similar_item_files[i], item_distances[i].min())
                   for i in range(test_images_normed.shape[1])]
for image_triplet in data_triplets_6:
    print_image_pair(*image_triplet)

Wie gut zu erkennen ist, wird in den meisten Faellen das richtige Bild als am aehnlichsten erkannt. Es gibt zwei
falsche Ergebnisse, bei denen aber deutlich ist, dass die Gesichter sehr aehnlich sind, obwohl es unterschiedliche
Personen sind.

Um die Performance analytisch zu evaluieren, wird nun eine Funktion definiert, welche fuer ein gegebenene
Test-Datensatz predictions erzeugt. Der Eigenspace wird ebenfalls mitgegeben, sodass unterschiedliche K-Werte getestet
werden koennen.

In [ ]:
def predict(train_images_normed, test_images_normed, eigenspace, train_image_names, test_image_names):
    def get_item_name(full_name):
        return full_name.split("/")[-1].split("-")[0]

    train_eigenspace = np.apply_along_axis(get_eigenspace_coordinates, 0, train_images_normed, eigenspace)
    test_eigenspace = np.apply_along_axis(get_eigenspace_coordinates, 0, test_images_normed, eigenspace)
    item_distances = np.apply_along_axis(get_distances, 0, test_eigenspace, train_eigenspace)
    most_similar_item_names = [train_image_names[get_most_similar_item_index(item_distances[:, i])] for i in
                               range(item_distances.shape[1])]
    y_actual = [get_item_name(test_image_name) for test_image_name in test_image_names]
    y_pred = [get_item_name(most_similar_item_name) for most_similar_item_name in most_similar_item_names]
    return y_actual, y_pred

Mithilfe dieser Funktion wird nun die Performance des Algorithmus fuer verschiedene K-Werte berechnet und ausgegeben.
Als Evaluierungsmass wird eine Confusion-Matrix, sowie ein Classification-Report fuer jeden K-Wert ausgegeben.

In [ ]:
k_configurations = [1, 5, 7, 10, 20, 50, 63]
predictions = {k_configuration: predict(
    train_images_normed, test_images_normed, get_most_relevant_eigenfaces(eigenfaces, k_configuration),
    train_image_files, test_image_files)
    for k_configuration in k_configurations}

for prediction in predictions.items():
    k, (y_actual, y_pred) = prediction
    print(f"Current K: {k}")
    cm = confusion_matrix(y_actual, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    fig, ax = plt.subplots(figsize=(10, 10))
    disp.plot(ax=ax)
    plt.show()
    print(classification_report(y_pred, y_actual))

Es ist gut zu erkennen, dass die Performance bis zu einem K-Wert von 7 start ansteigt. Danach gibt es erst wieder
bei einem K-Wert von 50 eine marginale Verbesserung in der Performance. Dies stimmt mit den experimentellen Ergebnissen
von *M. Turk, A. Pentland; Eigenfaces for Recognition* ueberein.

Um dies noch einmal zu veranschaulichen, wird der Verlauf der Precision ueber K geplottet.

In [ ]:
def get_precision(y_actual, y_pred):
    return sum([1 if y_actual[i] == y_pred[i] else 0 for i in range(len(y_actual))]) / len(y_actual)

precision_over_k = [get_precision(
    *predict(train_images_normed, test_images_normed,
            get_most_relevant_eigenfaces(eigenfaces, i + 1),
            train_image_files, test_image_files))
    for i in range(eigenfaces.shape[1])]

In [ ]:
DataFrame(precision_over_k).plot(title="Precision for K", ylabel="Precision",
                                 xlabel="K", figsize=(10, 10))

Wie man erkennen kann, steigt die Precision am mit niedrigen K-Werte start an. Bei 3, 4 und 6 scheint es eine leichte
Reduzierung der Precision zu geben, was darauf zurueckzuschliessen sein koennte, dass eine zusaetzliche Eigenspace-Dimension
dazu fuehrt, das ein bereits als richtig erkanntes Gesicht nun so abgefaelscht wird, dass es nicht mehr erkannt wird.
Ab einem K-Wert von 7 steigt die Precision nicht mehr weiter an. Erst ab einem K-Wert von etwa 28 wird noch einmal
ein zusaetzliches Gesicht korrekt erkannt.

Um zu testen, ob der Algorithmus auch Bilder, die keine Gesichter enthalten, als solche erkennt, wird nun ein Bild
von einem Hund geladen, und auf den Algorithmus angewandt.

Zuerst wird das Bild geladen und normalisiert.

In [ ]:
dog_image_file = join(".", "dog_resized_grayscale.jpg")
dog_image = read_images_to_numpy_data([dog_image_file])
dog_image_normalized = get_normed_images(dog_image, get_mean_image(train_images_raw))
dog_image_normalized.shape

In [ ]:
show_image(dog_image, 150, 220)


Nun wird das Bild in den Eigenspace transformiert.

In [ ]:
dog_image_eigenspace = get_eigenspace_coordinates(dog_image_normalized, eigenface_space)
dog_image_eigenspace

Um zu ermitteln ob es sich um ein menschliches Gesicht handelt, muss ein Threshold definiert werden, also eine
Distanz, ab der es nicht mehr als aehnlich angesehen wird.
Da das experimentelle Wissen dafuer fehlt, wird davon ausgegangen, dass der Threshold die maximale Minimaldistanz aus
der vorherigen Aufgabe ist.

#%

In [ ]:
threshold = np.max(np.min(item_distances, axis=0))
threshold

In [ ]:
def is_face(image_coordinates, comparison_faces, threshold):
    return np.min(get_distances(image_coordinates, comparison_faces)) < threshold

In [ ]:
print(f"Is dog a face: {is_face(dog_image_eigenspace, train_eigenspace, threshold)}")

Der Threshold sollte experimentell noch einmal genauer spezifiziert werden,
aber laut dem Threshold aus der vorherigen Aufgabe klassifiziert der Hund nicht als Gesicht.
